# Context 

H-1B visas are a category of employment-based, non-immigrant visas for temporary foreign workers in the United States. For a foreign national to apply for H1-B visa, a US employer must offer them a job and submit a petition for a H-1B visa to the US immigration department. This is also the most common visa status applied for and held by international students once they complete college or higher education and begin working in a full-time position.

The following articles contain more information about the H-1B visa process:

* [What is H1B LCA ? Why file it ? Salary, Processing times – DOL](https://redbus2us.com/what-is-h1b-lca-why-file-it-salary-processing-times-dol/)
* [H1B Application Process: Step by Step Guide](http://www.immi-usa.com/h1b-application-process-step-by-step-guide/)

# Content

This dataset contains five year's worth of H-1B petition data, with approximately 3 million records overall. The columns in the dataset include case status, employer name, worksite coordinates, job title, prevailing wage, occupation code, and year filed.

For more information on individual columns, refer to the column metadata. A detailed description of the underlying raw dataset is available in [an official data dictionary](https://www.foreignlaborcert.doleta.gov/docs/Performance_Data/Disclosure/FY15-FY16/H-1B_FY16_Record_Layout.pdf).

# Acknowledgements

The Office of Foreign Labor Certification (OFLC) generates program data, including data about H1-B visas. The disclosure data updated annually and is available [online](https://www.foreignlaborcert.doleta.gov/performancedata.cfm).

The raw data available is messy and not immediately suitable analysis. A set of data transformations were performed making the data more accessible for quick exploration. To learn more, refer to [this blog post](https://sharan-naribole.github.io/2017/02/24/h1b-eda-part-I.html) and to the complimentary [R Notebook](https://github.com/sharan-naribole/H1B_visa_eda/blob/master/data_processing.Rmd).

# Inspiration

* Is the number of petitions with Data Engineer job title increasing over time?
* Which part of the US has the most Hardware Engineer jobs?
* Which industry has the most number of Data Scientist positions?
* Which employers file the most petitions each year?

In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.types import BooleanType, LongType, ShortType, DoubleType
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We load a (local) SparkSession

In [2]:
spark = SparkSession.builder\
                    .master("local")\
                    .appName("Data Exploration With Spark")\
                    .getOrCreate()

## Loading the data

In [55]:
df = spark.read.load("data/h1b_kaggle.csv",
                     format="csv",
                     header="true",
                     inferSchema="true")

In [56]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- CASE_STATUS: string (nullable = true)
 |-- EMPLOYER_NAME: string (nullable = true)
 |-- SOC_NAME: string (nullable = true)
 |-- JOB_TITLE: string (nullable = true)
 |-- FULL_TIME_POSITION: string (nullable = true)
 |-- PREVAILING_WAGE: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- WORKSITE: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- lat: string (nullable = true)



Everything is string! We should definitely change this.

In [57]:
df = df.withColumn("FULL_TIME_POSITION", df["FULL_TIME_POSITION"].cast(BooleanType()))\
       .withColumn("PREVAILING_WAGE", df["PREVAILING_WAGE"].cast(LongType()))\
       .withColumn("YEAR", df["YEAR"].cast(ShortType()))\
       .withColumn("lon", df["lon"].cast(DoubleType()))\
       .withColumn("lat", df["lat"].cast(DoubleType()))

In [58]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- CASE_STATUS: string (nullable = true)
 |-- EMPLOYER_NAME: string (nullable = true)
 |-- SOC_NAME: string (nullable = true)
 |-- JOB_TITLE: string (nullable = true)
 |-- FULL_TIME_POSITION: boolean (nullable = true)
 |-- PREVAILING_WAGE: long (nullable = true)
 |-- YEAR: short (nullable = true)
 |-- WORKSITE: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- lat: double (nullable = true)



Now it makes more sense!

In [59]:
pd.DataFrame(df.take(10), columns=df.columns)

,_c0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,False,36067,2016,"ANN ARBOR, MICHIGAN",-83.743038,42.280826
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,True,242674,2016,"PLANO, TEXAS",-96.698886,33.019843
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,True,193066,2016,"JERSEY CITY, NEW JERSEY",-74.077642,40.728158
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",True,220314,2016,"DENVER, COLORADO",-104.990251,39.739236
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,True,157518,2016,"ST. LOUIS, MISSOURI",-90.199404,38.627003
5,6,CERTIFIED-WITHDRAWN,BURGER KING CORPORATION,CHIEF EXECUTIVES,"EXECUTIVE V P, GLOBAL DEVELOPMENT AND PRESIDEN...",True,225000,2016,"MIAMI, FLORIDA",-80.191790,25.761680
6,7,CERTIFIED-WITHDRAWN,BT AND MK ENERGY AND COMMODITIES,CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,True,91021,2016,"HOUSTON, TEXAS",-95.369803,29.760427
7,8,CERTIFIED-WITHDRAWN,"GLOBO MOBILE TECHNOLOGIES, INC.",CHIEF EXECUTIVES,CHIEF OPERATIONS OFFICER,True,150000,2016,"SAN JOSE, CALIFORNIA",-121.886329,37.338208
8,9,CERTIFIED-WITHDRAWN,ESI COMPANIES INC.,CHIEF EXECUTIVES,PRESIDENT,True,127546,2016,"MEMPHIS, TEXAS",NaN,NaN
9,10,WITHDRAWN,LESSARD INTERNATIONAL LLC,CHIEF EXECUTIVES,PRESIDENT,True,154648,2016,"VIENNA, VIRGINIA",-77.265260,38.901223


Are there any repeated rows?

In [79]:
df.count()

3002458

In [61]:
df.distinct().count()

3002458

It appearts all rows are unique. What about NA values?

In [64]:
df.na.drop().count()

2894761

May it be that most NA values come from the lat & lon columns?

In [71]:
for column in df.columns:
    print(column, df.na.drop(subset=[column]).count())

_c0 3002458
CASE_STATUS 3002458
EMPLOYER_NAME 3002458
SOC_NAME 3002458
JOB_TITLE 3002458
FULL_TIME_POSITION 3002413
PREVAILING_WAGE 3001967
YEAR 3002415
WORKSITE 3002458
lon 2895195
lat 2895209


Yes, it seems that most NA values come from the lat & lon columns, while a few hundreds of NA are in the PREVAILING_WAGE column.

In [84]:
pd.DataFrame(df.describe().take(20), columns=df.columns).transpose()

,0,1,2,3,4
_c0,count,mean,stddev,min,max
CASE_STATUS,3002458,1501229.5,866735.1116028279,1,3002458
EMPLOYER_NAME,3002458,None,None,CERTIFIED,WITHDRAWN
SOC_NAME,3002458,3.218588665E8,2.2409937148797384E8,"""""""EXCELLENT COMPUTING DISTRIBUTORS INC""""""","ËNIMAI, INC."
JOB_TITLE,3002458,None,None,"D/B/A/ THE ROHATYN GROUP""",Zoologists and Wildlife Biologists
FULL_TIME_POSITION,3002458,238095.35,2011997.0630112377,"""""""BUSINESS SYSTEM ANALYST""",TEST ANALYST - US
PREVAILING_WAGE,3001967,146924.89643923467,5287283.137188399,0,6997606720
YEAR,3002415,2013.8550280357645,1.6806138144451777,2011,2016
WORKSITE,3002458,1523.8567095766666,901.9284495964259,"""O""""FALLEN","FORT WASHINGTON, PENNSYLVANIA"
lon,2895195,-92.1342144142733,19.65698073584893,-157.8583333,145.7297891


Let's start with taking a look at the H1Bs by the status of their visa applications.

In [128]:
pd.DataFrame(df.select("CASE_STATUS").distinct().sort("CASE_STATUS").take(10))

,0
0,CERTIFIED
1,CERTIFIED-WITHDRAWN
2,DENIED
3,INVALIDATED
4,NA
5,PENDING QUALITY AND COMPLIANCE REVIEW - UNASSI...
6,REJECTED
7,WITHDRAWN


In [137]:
df.filter(df.CASE_STATUS == "NA").count()

13

Cross Tabulation provides a table of the frequency distribution for a set of variables.

In [134]:
pd.DataFrame(df.crosstab('EMPLOYER_NAME', 'CASE_STATUS').take(20), columns=["EMPLOYER_NAME", "CERTIFIED", "CERTIFIED-WITHDRAWN", "DENIED", "INVALIDATED", "NA", "PENDING REVIEW", "REJECTED", "WITHDRAWN"])

,EMPLOYER_NAME,CERTIFIED,CERTIFIED-WITHDRAWN,DENIED,INVALIDATED,NA,PENDING REVIEW,REJECTED,WITHDRAWN
0,"PATHWAY BIOLOGIC, LLC",1,0,0,0,0,0,0,0
1,"TCL RESEARCH AMERICA, INC.",14,0,0,0,0,0,0,1
2,UNIVERSITY OF MAINE,97,1,3,0,0,0,0,5
3,P2F HOLDINGS,2,0,0,0,0,0,0,0
4,"M2S, INC.",11,0,0,0,0,0,0,0
5,NANOVIRICIDES INC,1,0,2,0,0,0,0,0
6,"ARIZONA CANCER SPECIALISTS, PLC",1,0,0,0,0,0,0,0
7,"CLARITY MONEY, INC.",1,0,0,0,0,0,0,0
8,W.R. BERKLEY CORPORATION,5,0,0,0,0,0,0,2
9,S. VINODKUMAR USA INC.,4,0,0,0,0,0,0,0


If you are comfortable with SQL queries, you can use them straight away if you create a Global Temporary View
first.

Top10 companies getting visa approval (for all the years):

In [151]:
df.createGlobalTempView("df")

In [152]:
spark.sql("SELECT EMPLOYER_NAME, count(EMPLOYER_NAME) as CERTIFIED_COUNT FROM df where CASE_STATUS = 'CERTIFIED' GROUP BY EMPLOYER_NAME order by CERTIFIED_COUNT desc").show(10)

+--------------------+---------------+
|       EMPLOYER_NAME|CERTIFIED_COUNT|
+--------------------+---------------+
|     INFOSYS LIMITED|         129916|
|TATA CONSULTANCY ...|          64237|
|       WIPRO LIMITED|          43476|
|DELOITTE CONSULTI...|          36120|
|       ACCENTURE LLP|          32911|
|IBM INDIA PRIVATE...|          27745|
|MICROSOFT CORPORA...|          22333|
|   HCL AMERICA, INC.|          22234|
|ERNST & YOUNG U.S...|          17874|
|LARSEN & TOUBRO I...|          16652|
+--------------------+---------------+
only showing top 10 rows



Top10 companies getting visa approval (for year 2016)

In [154]:
spark.sql("SELECT EMPLOYER_NAME, count(EMPLOYER_NAME) as CERTIFIED_COUNT FROM df where CASE_STATUS = 'CERTIFIED' AND YEAR='2016' GROUP BY EMPLOYER_NAME order by CERTIFIED_COUNT desc").show(10)

+--------------------+---------------+
|       EMPLOYER_NAME|CERTIFIED_COUNT|
+--------------------+---------------+
|     INFOSYS LIMITED|          25322|
|CAPGEMINI AMERICA...|          15957|
|TATA CONSULTANCY ...|          13072|
|       WIPRO LIMITED|           9528|
|       ACCENTURE LLP|           9374|
|IBM INDIA PRIVATE...|           7824|
|DELOITTE CONSULTI...|           7500|
|TECH MAHINDRA (AM...|           6681|
|   HCL AMERICA, INC.|           4917|
|MICROSOFT CORPORA...|           4669|
+--------------------+---------------+
only showing top 10 rows



Worksites for which most number of visas are approved or certified

In [155]:
spark.sql("SELECT WORKSITE, count(CASE_STATUS) as APPROVED_COUNT FROM df where CASE_STATUS = 'CERTIFIED' GROUP BY WORKSITE order by APPROVED_COUNT desc").show(10)

+--------------------+--------------+
|            WORKSITE|APPROVED_COUNT|
+--------------------+--------------+
|  NEW YORK, NEW YORK|        163982|
|      HOUSTON, TEXAS|         71540|
|SAN FRANCISCO, CA...|         54336|
|    ATLANTA, GEORGIA|         46877|
|   CHICAGO, ILLINOIS|         45249|
|SAN JOSE, CALIFORNIA|         44305|
|SUNNYVALE, CALIFO...|         30294|
|CHARLOTTE, NORTH ...|         27966|
|       DALLAS, TEXAS|         27810|
| REDMOND, WASHINGTON|         27419|
+--------------------+--------------+
only showing top 10 rows



TOP 5 JOB TITLE for which visa are approved

In [160]:
spark.sql("SELECT JOB_TITLE, count(CASE_STATUS) as VISAS_APPROVED FROM df where CASE_STATUS = 'CERTIFIED' GROUP BY JOB_TITLE order by VISAS_APPROVED desc").show(10)

+--------------------+--------------+
|           JOB_TITLE|VISAS_APPROVED|
+--------------------+--------------+
|  PROGRAMMER ANALYST|        222730|
|   SOFTWARE ENGINEER|        102990|
| COMPUTER PROGRAMMER|         64018|
|     SYSTEMS ANALYST|         55744|
|  SOFTWARE DEVELOPER|         37771|
|    BUSINESS ANALYST|         35202|
|COMPUTER SYSTEMS ...|         31433|
|TECHNOLOGY LEAD - US|         28280|
|TECHNOLOGY ANALYS...|         25985|
|SENIOR SOFTWARE E...|         23547|
+--------------------+--------------+
only showing top 10 rows



In [161]:
spark.stop()

AttributeError: 'SparkSession' object has no attribute 'quit'